##  Gradient Boosting and AdaBoosting Algorithms


### Gradient Boosting Implementation

Gradient Boosting is a powerful technique where new models are trained to correct the errors of the previous models in an iterative fashion.

In this task, we'll:
1. Implement Gradient Boosting from scratch using NumPy.
2. Use scikit-learn's GradientBoostingClassifier for comparison.
3. Experiment with hyperparameters such as learning rate, number of estimators, and maximum depth to optimize the model's performance.
    

In [1]:

# Import libraries
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

# Gradient Boosting from scratch for regression
class GradientBoostingRegressorScratch:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []
        
    def fit(self, X, y):
        # Initial prediction is the mean of the target
        self.initial_pred = np.mean(y)
        residuals = y - self.initial_pred
        
        for _ in range(self.n_estimators):
            model = DecisionTreeRegressor(max_depth=self.max_depth)
            model.fit(X, residuals)
            self.models.append(model)
            residuals -= self.learning_rate * model.predict(X)
    
    def predict(self, X):
        y_pred = np.full(X.shape[0], self.initial_pred)
        for model in self.models:
            y_pred += self.learning_rate * model.predict(X)
        return y_pred

# Generate synthetic data for regression
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# GradientBoostingClassifier using scikit-learn
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = gb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Gradient Boosting Classifier Accuracy: {accuracy}')

# Experiment with hyperparameters
learning_rates = [0.01, 0.1, 0.2, 0.5]
for lr in learning_rates:
    gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=lr, max_depth=3, random_state=42)
    gb_model.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Learning Rate: {lr}, Accuracy: {accuracy}')


Gradient Boosting Classifier Accuracy: 0.8833333333333333
Learning Rate: 0.01, Accuracy: 0.8
Learning Rate: 0.1, Accuracy: 0.8833333333333333
Learning Rate: 0.2, Accuracy: 0.9066666666666666
Learning Rate: 0.5, Accuracy: 0.8866666666666667



### AdaBoost Implementation

AdaBoost trains a series of weak learners (typically decision trees with only one split) and combines them into a strong classifier.

In this task, we'll:
1. Implement AdaBoost from scratch.
2. Use scikit-learn's AdaBoostClassifier and compare results.
3. Use different weak learners and evaluate their impact on boosting performance.
    

In [2]:

# Import libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# AdaBoost from scratch
class AdaBoostScratch:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.models = []
        self.alphas = []
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        w = np.full(n_samples, 1 / n_samples)
        
        for _ in range(self.n_estimators):
            model = DecisionTreeClassifier(max_depth=1)
            model.fit(X, y, sample_weight=w)
            predictions = model.predict(X)
            error = np.sum(w * (predictions != y)) / np.sum(w)
            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))
            self.models.append(model)
            self.alphas.append(alpha)
            w *= np.exp(-alpha * y * predictions)
            w /= np.sum(w)
    
    def predict(self, X):
        final_pred = np.zeros(X.shape[0])
        for alpha, model in zip(self.alphas, self.models):
            final_pred += alpha * model.predict(X)
        return np.sign(final_pred)

# Generate synthetic data for classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, random_state=42)
y[y == 0] = -1  # Convert to {-1, 1} for AdaBoost

# AdaBoostClassifier using scikit-learn
ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, random_state=42)
ada_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = ada_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'AdaBoost Classifier Accuracy: {accuracy}')

# Experiment with different weak learners
base_learners = [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)]
for learner in base_learners:
    ada_model = AdaBoostClassifier(base_estimator=learner, n_estimators=50, random_state=42)
    ada_model.fit(X_train, y_train)
    y_pred = ada_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Base Learner Depth: {learner.max_depth}, Accuracy: {accuracy}')


AdaBoost Classifier Accuracy: 0.8033333333333333
Base Learner Depth: 1, Accuracy: 0.8033333333333333
Base Learner Depth: 2, Accuracy: 0.8733333333333333
Base Learner Depth: 3, Accuracy: 0.8466666666666667
